In [1]:
# mount driver
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CardiacSeg/SwinUNETR/Pretrain/Pretrain

# install dependents
!pip install -q nnunet
!pip install -q "monai-weekly[nibabel, tqdm, einops]"
!pip install -q tensorboardX==2.1
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

# sync python module
%load_ext autoreload
%autoreload 2

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1jz_DGnICBmKWCr_JL904PDQdIEK0_EQG/CardiacSeg/SwinUNETR/Pretrain/Pretrain
     |████████████████████████████████| 251 kB 24.7 MB/s 
     |████████████████████████████████| 43 kB 1.4 MB/s 
     |████████████████████████████████| 151 kB 57.3 MB/s 
     |████████████████████████████████| 61 kB 331 kB/s 
     |████████████████████████████████| 52.8 MB 256 kB/s 
     |████████████████████████████████| 96 kB 432 kB/s 
     |████████████████████████████████| 13.0 MB 58.8 MB/s 
     |████████████████████████████████| 2.0 MB 53.0 MB/s 
     |████████████████████████████████| 1.1 MB 30.3 MB/s 


In [ ]:
import sys
# set package path
sys.path.append("/content/drive/MyDrive/CardiacSeg")

import os
from pathlib import PurePath

from monai.transforms import (
    Compose,
    LoadImaged,
)

from data_utils.json_dataset import generate_dataset_json
from data_utils import (
    segthor_dataset as sg_ds,
    mmwhs_dataset as mmwhs_ds
)

## Config

In [ ]:
sg_dir = '/content/drive/MyDrive/CardiacSeg/dataset/SegTHOR'
sg_data_dir = os.path.join(sg_dir, 'train')
sg_data_json_pth = os.path.join(sg_dir, 'dataset.json')

mmwhs_dir = '/content/drive/MyDrive/CardiacSeg/dataset/MM_WHS_2017'
mmwhs_data_dir = os.path.join(mmwhs_dir, 'ct_train')
mmwhs_data_json_pth = os.path.join(mmwhs_dir, 'dataset.json')

## Generate data json file

In [ ]:
# segthor
sg_data_dicts = sg_ds.get_data_dicts(sg_data_dir)
generate_dataset_json(
    sg_data_dicts,
    sg_data_json_pth,
    dataset_name='segthor',
    labels={0: 'background', 2: 'heart'},
    tensorImageSize='3D',
    sort_keys=True,
    modalities=['CT'],
    license='see challenge website',
    split_train_ratio=0.8,
    fold=4,
    num_fold=5,
)

fold: 4
train files (26): ['Patient_01', 'Patient_02', 'Patient_03', 'Patient_04', 'Patient_05', 'Patient_06', 'Patient_07', 'Patient_08', 'Patient_09', 'Patient_10', 'Patient_11', 'Patient_12', 'Patient_13', 'Patient_14', 'Patient_15', 'Patient_16', 'Patient_17', 'Patient_18', 'Patient_19', 'Patient_20', 'Patient_21', 'Patient_22', 'Patient_23', 'Patient_24', 'Patient_25', 'Patient_26']
val files (6): ['Patient_27', 'Patient_28', 'Patient_29', 'Patient_30', 'Patient_31', 'Patient_32']
test files (8): ['Patient_33', 'Patient_34', 'Patient_35', 'Patient_36', 'Patient_37', 'Patient_38', 'Patient_39', 'Patient_40']
save json to /content/drive/MyDrive/CardiacSeg/dataset/SegTHOR/dataset.json


In [ ]:
# mmwhs
mmwhs_data_dicts = mmwhs_ds.get_data_dicts(mmwhs_data_dir)
generate_dataset_json(
    mmwhs_data_dicts,
    mmwhs_data_json_pth,
    dataset_name='mmwhs',
    labels={
        0: 'background', 
        500: 'lv', 
        600: 'rv', 
        420: 'la', 
        550: 'ra', 
        205: 'mlv',
        820: 'aa',
        850: 'pa',
    },
    tensorImageSize='3D',
    sort_keys=True,
    modalities=['CT'],
    license='see challenge website',
    split_train_ratio=0.8,
    fold=4,
    num_fold=5,
)

fold: 4
train files (13): ['ct_train_1001_image', 'ct_train_1002_image', 'ct_train_1003_image', 'ct_train_1004_image', 'ct_train_1005_image', 'ct_train_1006_image', 'ct_train_1007_image', 'ct_train_1008_image', 'ct_train_1009_image', 'ct_train_1010_image', 'ct_train_1011_image', 'ct_train_1012_image', 'ct_train_1013_image']
val files (3): ['ct_train_1014_image', 'ct_train_1015_image', 'ct_train_1016_image']
test files (4): ['ct_train_1017_image', 'ct_train_1018_image', 'ct_train_1019_image', 'ct_train_1020_image']
save json to /content/drive/MyDrive/CardiacSeg/dataset/MM_WHS_2017/dataset.json


## Pretrain

In [ ]:
!python main.py\
--use_checkpoint \
--batch_size=1 \
--num_steps=100000 \
--lrdecay \
--eval_num=100 \
--logdir='exp_0'
# --lr=<Lr> 
# --roi_x=<Roi_x> \
# --roi_y=<Roi_y> \
# --roi_z=<Roi_z> \

Training with a single process on 1 GPUs.
load json from /content/drive/MyDrive/CardiacSeg/dataset/SegTHOR/dataset.json
load json from /content/drive/MyDrive/CardiacSeg/dataset/MM_WHS_2017/dataset.json
load json from /content/drive/MyDrive/CardiacSeg/dataset/SegTHOR/dataset.json
load json from /content/drive/MyDrive/CardiacSeg/dataset/MM_WHS_2017/dataset.json
Dataset 1 SegTHOR: number of tr data: 26
Dataset 2 MMWHS: number of tr data: 13
Dataset 1 SegTHOR: number of val data: 6
Dataset 2 MMWHS: number of val data: 3
Dataset all training: number of data: 39
Dataset all validation: number of data: 9
/usr/local/lib/python3.7/dist-packages/monai/utils/deprecate_utils.py:107: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
  warn_deprecated(obj, msg, warning_category)
Using generic dataset
/usr/local/lib/python3.7/dist-packages/torch/